### Work By Sherlyn, Xianyuan Wu, Xiaoyu Zhang

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

data1 = pd.read_csv("model1.csv")
data2 = pd.read_csv("model2.csv")
data3 = pd.read_csv("model3.csv")

data1['Date'] = pd.to_datetime(data1.Date, format='%Y-%m-%d')
data2['Date'] = pd.to_datetime(data2.Date, format='%Y-%m-%d')
data3['Date'] = pd.to_datetime(data3.Date, format='%Y-%m-%d')

data1 = data1[~data1[['Date', 'GVKEY']].duplicated(keep='last')]
data2 = data2[~data2[['Date', 'GVKEY']].duplicated(keep='last')]
data3 = data3[~data3[['Date', 'GVKEY']].duplicated(keep='last')]


data1["Return"] = data1["Return"]/100
data2["Return"] = data2["Return"]/100
data3["Return"] = data3["Return"]/100

data1_pivot =\
(data1.sort_values(['Date', 'GVKEY'])
      .pivot(index = 'Date', columns = 'GVKEY')
      .swaplevel(axis = 1).sort_index(axis = 1)
      .reindex(data1.columns[2:], axis = 1, level = 1))
data2_pivot =\
(data2.sort_values(['Date', 'GVKEY'])
      .pivot(index = 'Date', columns = 'GVKEY')
      .swaplevel(axis = 1).sort_index(axis = 1)
      .reindex(data2.columns[2:], axis = 1, level = 1))
data3_pivot =\
(data3.sort_values(['Date', 'GVKEY'])
      .pivot(index = 'Date', columns = 'GVKEY')
      .swaplevel(axis = 1).sort_index(axis = 1)
      .reindex(data3.columns[2:], axis = 1, level = 1))

# Question 1
### Monthly cross-sectional regression estimates for three models

#### Model 1

In [2]:
data1_pivot_cleaned = data1_pivot.copy()
data1_pivot_cleaned = data1_pivot_cleaned.loc["1969-09-30":,:]

def olsestimates(row):
    #column_names = ["constant","LogSize_-1 coefficient", "LogB/M_-1 coefficient", "Return_-2,-12 coefficient",
                   #"Adjusted R\u00b2", "Number of Firms or Obs"]
    #combined_dataframe = pd.DataFrame(columns = column_names)
    row.dropna(inplace = True)
    df_interim = row.unstack()
    X = sm.add_constant(df_interim.loc[:,"LogSize_-1":])
    Y = df_interim["Return"]
    model = sm.OLS(Y, X).fit()
    constant = round(model.params[0], 4)
    LS_coeff = round(model.params[1], 4)
    LB_coeff = round(model.params[2], 4)
    return_2_12_coeff = model.params[3]
    adjusted_rsq = model.rsquared_adj
    no_of_firms = model.nobs
    #f_value, f_pvalue = model.fvalue, model.f_pvalue
    
    return [constant, LS_coeff,LB_coeff,return_2_12_coeff, adjusted_rsq,no_of_firms]
    

test_model1 = data1_pivot_cleaned.apply(olsestimates, axis = 1)
test_model1_list = test_model1.to_list()
test_model1_df = pd.DataFrame(test_model1_list, columns = ["constant","LogSize_-1 coefficient", "LogB/M_-1 coefficient", "Return_-2,-12 coefficient",
                                       "Adjusted R\u00b2", "Number of Firms or Obs"])
test_model1_df.insert(0,"Date", data1_pivot_cleaned.index)
test_model1_df.set_index("Date", inplace = True)

#### Model 2

In [3]:
def olsestimates2(row):
    #column_names = ["constant","LogSize_-1 coefficient", "LogB/M_-1 coefficient", "Return_-2,-12 coefficient",
                   #"Adjusted R\u00b2", "Number of Firms or Obs"]
    #combined_dataframe = pd.DataFrame(columns = column_names)
    row.dropna(inplace = True)
    df_interim = row.unstack()
    X = sm.add_constant(df_interim.loc[:,"LogSize_-1":])
    Y = df_interim["Return"]
    model = sm.OLS(Y, X).fit()
    constant = model.params[0]
    LS_coeff = model.params[1]
    LB_coeff = model.params[2]
    return_2_12_coeff = model.params[3]
    logissues = model.params[4]
    accruals = model.params[5]
    roa = model.params[6]
    logAGY = model.params[7]
    adjusted_rsq = model.rsquared_adj
    no_of_firms = model.nobs
    #f_value, f_pvalue = model.fvalue, model.f_pvalue
    
    return [constant, LS_coeff,LB_coeff,return_2_12_coeff, logissues, accruals,
            roa, logAGY, adjusted_rsq,no_of_firms]

data2_pivot_cleaned = data2_pivot.copy()
data2_pivot_cleaned = data2_pivot_cleaned.loc["1969-09-30":,:]

test_model2 = data2_pivot_cleaned.apply(olsestimates2, axis = 1)
test_model2_list = test_model2.to_list()
test_model2_df = pd.DataFrame(test_model2_list, columns = ["constant","LogSize_-1 coefficient", "LogB/M_-1 coefficient", "Return_-2,-12 coefficient", "Logissues-1,-36 coefficient",
                                                           "AccrualsYr-1 coefficient","ROAYr-1 coefficient", "LogAGYr-1 coefficient",
                                                           "Adjusted R\u00b2", "Number of Firms or Obs"])
test_model2_df.insert(0,"Date", data2_pivot_cleaned.index)
test_model2_df.set_index("Date", inplace = True)

#### Model 3 

In [4]:
def olsestimates3(row):
    #column_names = ["constant","LogSize_-1 coefficient", "LogB/M_-1 coefficient", "Return_-2,-12 coefficient",
                   #"Adjusted R\u00b2", "Number of Firms or Obs"]
    #combined_dataframe = pd.DataFrame(columns = column_names)
    row.dropna(inplace = True)
    df_interim = row.unstack()
    X = sm.add_constant(df_interim.loc[:,"LogSize_-1":])
    Y = df_interim["Return"]
    model = sm.OLS(Y, X).fit()
    constant = model.params[0]
    LS_coeff = model.params[1]
    LB_coeff = model.params[2]
    return_2_12_coeff = model.params[3]
    logissues_1_36 = model.params[4]
    accruals = model.params[5]
    roa = model.params[6]
    logAGY = model.params[7]
    dy_1 = model.params[8]
    logreturn_13_36 = model.params[9]
    logissues_1_12 = model.params[10]
    turnover = model.params[11]
    debt_price = model.params[12]
    sales_price = model.params[13]
    adjusted_rsq = model.rsquared_adj
    no_of_firms = model.nobs
    #f_value, f_pvalue = model.fvalue, model.f_pvalue
    
    return [constant, LS_coeff,LB_coeff,return_2_12_coeff, logissues_1_36, accruals,
            roa, logAGY, dy_1, logreturn_13_36, logissues_1_12, turnover,debt_price,sales_price,
            adjusted_rsq,no_of_firms]
    

data3_pivot_cleaned = data3_pivot.copy()
data3_pivot_cleaned = data3_pivot_cleaned.loc["1969-09-30":,:]

test_model3 = data3_pivot_cleaned.apply(olsestimates3, axis = 1)
test_model3_list = test_model3.to_list()
test_model3_df = pd.DataFrame(test_model3_list, columns = ["constant","LogSize_-1 coefficient", "LogB/M_-1 coefficient", "Return_-2,-12 coefficient", "Logissues-1,-36 coefficient",
                                                           "AccrualsYr-1 coefficient","ROAYr-1 coefficient", "LogAGYr-1 coefficient", "DY-1,-12 coefficient","LogReturn-13,-36 coefficient",
                                                           "Logissues-1,-12 coefficient", "Turnover-1,-12 coefficient", "Debt/PriceYr-1 coefficient","Sales/PriceYr-1 coefficient",
                                                           "Adjusted R\u00b2", "Number of Firms or Obs"])
test_model3_df.insert(0,"Date", data3_pivot_cleaned.index)
test_model3_df.set_index("Date", inplace = True)

#test_model1_df.to_excel('Part1_M1.xlsx', sheet_name = 'Model 1')
#test_model2_df.to_excel('Part1_M2.xlsx', sheet_name = 'Model 2')
#test_model3_df.to_excel('Part1_M3.xlsx', sheet_name = 'Model 3')

# Question 2
### T-test for the time series average of the slope estimates (risk premium estimates each month)

In [5]:
def obtain_mean_se_t_stats(df):
    new_df = df.iloc[:,:-2]
    average_slope_df = pd.DataFrame(new_df.mean(axis = 0), columns = ["Average"])
    standard_error_df = pd.DataFrame(new_df.sem(axis =0), columns = ["Standard Error"])
    t_test = stats.ttest_1samp(new_df,0, axis = 0, alternative = "two-sided")
    t_test_df = (pd.DataFrame(t_test)).T
    t_test_df.set_index(new_df.columns, inplace = True)
    t_test_df.rename(columns = {0: "t_statistic",1:"P_value"}, inplace = True)
    new_df2 = pd.concat([average_slope_df, t_test_df], axis = 1)
    significance_indicator = new_df2["P_value"].apply(lambda x: "***" if x < 0.01 else "**" if x < 0.05 else "*" if x < 0.1 else " ")
    new_df2["Significance Level"] = significance_indicator
    return new_df2
                                            

def getFormat(df_list):
    form_list = []
    for i in range(3):
        form = obtain_mean_se_t_stats(df_list[i])
        form['Average'] = form['Average'].astype(str).add(form['Significance Level'])
        form = pd.DataFrame(form[['Average', 't_statistic']].stack())
        form.rename(columns = {0: 'Model ' + str(int(i)+1)}, inplace = True)
        form_list.append(form)
        result = pd.concat(form_list, axis = 1, join = 'outer')
        index = [('constant','Average'), ('constant','t_statistic'), 
         ('LogSize_-1 coefficient','Average'), ('LogSize_-1 coefficient','t_statistic'),
         ('LogB/M_-1 coefficient','Average'), ('LogB/M_-1 coefficient','t_statistic'),
         ('Return_-2,-12 coefficient','Average'),('Return_-2,-12 coefficient','t_statistic'),
         ("Logissues-1,-36 coefficient",'Average'),('Logissues-1,-36 coefficient','t_statistic'),
         ("AccrualsYr-1 coefficient",'Average'), ('AccrualsYr-1 coefficient','t_statistic'),
         ("ROAYr-1 coefficient",'Average'), ('ROAYr-1 coefficient','t_statistic'),
         ("LogAGYr-1 coefficient",'Average'),('LogAGYr-1 coefficient','t_statistic'),
         ("DY-1,-12 coefficient",'Average'),('DY-1,-12 coefficient','t_statistic'),
         ("LogReturn-13,-36 coefficient",'Average'),('LogReturn-13,-36 coefficient','t_statistic'),
         ("Logissues-1,-12 coefficient",'Average'),('Logissues-1,-12 coefficient','t_statistic'),
         ("Turnover-1,-12 coefficient",'Average'),('Turnover-1,-12 coefficient','t_statistic'),
         ("Debt/PriceYr-1 coefficient",'Average'),('Debt/PriceYr-1 coefficient','t_statistic'),
         ("Sales/PriceYr-1 coefficient",'Average'),('Sales/PriceYr-1 coefficient','t_statistic'),]
        result = result.reindex(index)
    return result

result2 = getFormat([test_model1_df, test_model2_df, test_model3_df])

#result2.to_excel('part2.xlsx')

# Question 3
### Forecasting using model 3

In [6]:
m3 = test_model3_df['1970-09-30':].reset_index()

mean_list = []
for i in range(len(m3)-119):
    mean = m3.iloc[i: i+120, 2:-2].mean().to_list()
    mean_list.append(mean)
m3_ols = pd.DataFrame(mean_list, index = m3['Date'][119:], columns = [["LogSize_-1 coefficient", "LogB/M_-1 coefficient", "Return_-2,-12 coefficient", "Logissues-1,-36 coefficient",
                                                        "AccrualsYr-1 coefficient","ROAYr-1 coefficient", "LogAGYr-1 coefficient", "DY-1,-12 coefficient","LogReturn-13,-36 coefficient",
                                                       "Logissues-1,-12 coefficient", "Turnover-1,-12 coefficient", "Debt/PriceYr-1 coefficient","Sales/PriceYr-1 coefficient"]])

m3_gamma = m3_ols.reset_index()

loading_matrix = data3_pivot_cleaned['1980-08-31':]

from pandas_datareader import data as pdr

start = '1980-07-31'
end = '2021-03-31'
drange=pd.DataFrame(index=m3_ols.index)

rf=pdr.get_data_fred("TB3MS", start=start, end=end)
#rf=pdr.get_data_fred("DTB4WK", start=start, end=end)
rf=rf/100
rf.rename(columns={"TB3MS":"rf_rate"}, inplace=True)
#rf.rename(columns={"DTB4WK":"rf_rate"}, inplace=True)


def getmultiply(risk_pre, loading_mx, rf_vec):
    no_positive = []
    no_negative = []
    total = []
    for i in range(len(risk_pre)-1):
        loading = loading_mx.iloc[i].dropna().unstack()
        loading_pro = loading_mx.iloc[i+1].dropna().unstack()
        product = np.multiply(loading.iloc[:, 1:], risk_pre.iloc[i, 1:]).sum(axis = 1) + rf_vec.iloc[i][0] - loading['Return']
        actual_distance = loading_pro['Return'] - loading['Return'] #return distance at date i+1
        compare = np.multiply(product, actual_distance).dropna() #n times 1 matrix
        total.append(compare.count())
        p = 0
        n = 0
        for j in compare:
            if j < 0:
                n += 1
            elif j > 0:
                p += 1
        no_positive.append(p)
        no_negative.append(n)
        
    no_positive.append(0)
    no_negative.append(0)
    total.append(0)
    result = pd.DataFrame([no_positive, no_negative, total], index = [['No. of +1’s', 'No. of -1’s', 'Total Number of Stocks']], 
                          columns = m3_ols.index)
    return result.T

exp_r = getmultiply(m3_gamma, loading_matrix, rf)

result3 = exp_r.shift(1).dropna().astype(int)

accuracy_df = result3.copy()
accuracy_df['Accuracy Percentage']= np.multiply(accuracy_df['No. of +1’s'], 1/accuracy_df['Total Number of Stocks'])
#accuracy_df.to_excel('Part3.xlsx')
accuracy_df

<ipython-input-6-58970c6b77bb>:35: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  product = np.multiply(loading.iloc[:, 1:], risk_pre.iloc[i, 1:]).sum(axis = 1) + rf_vec.iloc[i][0] - loading['Return']
<ipython-input-6-58970c6b77bb>:61: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the a

,No. of +1’s,No. of -1’s,Total Number of Stocks,Accuracy Percentage
Date,,,,
1980-09-30,1232,509,1745,0.706017
1980-10-31,1075,652,1732,0.620670
1980-11-30,1126,578,1709,0.658865
1980-12-31,856,825,1682,0.508918
1981-01-31,1024,613,1641,0.624010
...,...,...,...,...
2020-11-30,1170,617,1787,0.654729
2020-12-31,1395,384,1779,0.784148
2021-01-31,459,124,583,0.787307
